## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Domino Data Lab Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. You need to make sure that these datasets are uploaded and present in the same directory as this notebook file, since we will be importing these files in Spark and create the following tables under the __`country_club`__ database using Spark SQL.

1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into Spark to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the Domino platform, but for the final deliverable, please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

# Checking Existence of Spark Environment Variables

Make sure your notebook is loaded using a PySpark Workspace. If you open up a regular Jupyter workspace the following variables might not exist

In [2]:
spark

NameError: name 'spark' is not defined

In [ ]:
sqlContext

### Run the following if you failed to open a notebook in the PySpark Workspace

This will work assuming you are using Spark in the cloud on domino or you might need to configure with your own spark instance if you are working offline

In [3]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

# Create a utility function to run SQL commands

Instead of typing the same python functions repeatedly, we build a small function where you can just pass your query to get results.

- Remember we are using Spark SQL in PySpark
- We can't run multiple SQL statements in one go (no semi-colon ';' separated SQL statements)
- We can run multi-line SQL queries (but still has to be a single statement)

In [4]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

# Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [5]:
run_sql('drop database if exists country_club cascade')
run_sql('create database country_club')
dbs = run_sql('show databases')
dbs.toPandas()

,databaseName
0,country_club
1,default


# Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you.

To get started, first make sure you have already uploaded the three CSV files and they are present in the same directory as the notebook.

Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [6]:
# File location and type
file_location_bookings = "./Bookings.csv"
file_location_facilities = "./Facilities.csv"
file_location_members = "./Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [7]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



# Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [8]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

AnalysisException: "Can not create the managed table('`country_club`.`Bookings`'). The associated location('file:/Users/jzmichal/Downloads/springboard-aic-mini-projects-public/aic-5_6_6-sql-at-scale-with-spark-mini-project/spark-warehouse/country_club.db/bookings') already exists.;"

### Refresh tables and check them

In [9]:
run_sql('use country_club')
run_sql('REFRESH table bookings')
run_sql('REFRESH table facilities')
run_sql('REFRESH table members')
tbls = run_sql('show tables')
tbls.toPandas()

Table or view 'bookings' not found in database 'country_club'; 
 org.apache.spark.sql.catalyst.catalog.SessionCatalog.requireTableExists(SessionCatalog.scala:185)
	 at org.apache.spark.sql.catalyst.catalog.SessionCatalog.getTableMetadata(SessionCatalog.scala:433)
	 at org.apache.spark.sql.catalyst.catalog.SessionCatalog$$anonfun$getTempViewOrPermanentTableMetadata$2.apply(SessionCatalog.scala:586)
	 at org.apache.spark.sql.catalyst.catalog.SessionCatalog$$anonfun$getTempViewOrPermanentTableMetadata$2.apply(SessionCatalog.scala:586)
	 at scala.Option.getOrElse(Option.scala:121)
	 at org.apache.spark.sql.catalyst.catalog.SessionCatalog.getTempViewOrPermanentTableMetadata(SessionCatalog.scala:586)
	 at org.apache.spark.sql.internal.CatalogImpl.refreshTable(CatalogImpl.scala:481)
	 at org.apache.spark.sql.execution.datasources.RefreshTable.run(ddl.scala:111)
	 at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	 at org.apache.spark.

,database,tableName,isTemporary


# Test a sample SQL query

__Note:__ You can use multi-line SQL queries (but still a single statement) as follows

In [ ]:
result = run_sql('''
                    SELECT * 
                    FROM bookings
                    LIMIT 3
                 ''')
result.toPandas()

# Your Turn: Solve the following questions with Spark SQL

- Make use of the `run_sql(...)` function as seen in the previous example
- You can write multi-line SQL queries but it has to be a single statement (no use of semi-colons ';')
- Make use of the `toPandas()` function as depicted in the previous example to display the query results

#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [ ]:
result = run_sql('''
                    SELECT name
                    FROM facilities
                    WHERE membercost > 0
                 ''')
result.toPandas()

####  Q2: How many facilities do not charge a fee to members?

In [ ]:
result = run_sql('''
                    SELECT count(name) as number_of_facilities_with_no_fee
                    FROM facilities
                    WHERE membercost == 0
                 ''')
result.toPandas()

#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [ ]:
result = run_sql('''
                    SELECT facid, name, membercost, monthlymaintenance
                    FROM facilities
                    WHERE membercost / monthlymaintenance < .2
                ''')
result.toPandas()

#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [ ]:
result = run_sql('''
                    SELECT * 
                    FROM facilities
                    WHERE facid IN (1,5)
                ''')
result.toPandas()

#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [ ]:
result = run_sql('''
                   SELECT name, monthlymaintenance,
                       CASE 
                           WHEN monthlymaintenance <= 100 THEN "cheap"
                           ELSE "expensive"
                        END AS pricing_category
                   FROM facilities
                   
                 ''')
result.toPandas()

#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [ ]:
result = run_sql('''
                    SELECT firstname, surname
                    FROM members
                    WHERE joindate = (SELECT MAX(joindate) FROM members)
                 ''')
result.toPandas()

####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [ ]:
result = run_sql('''
                   SELECT DISTINCT firstname, surname, name
                   FROM members
                   INNER JOIN bookings 
                    USING (memid)
                    INNER JOIN facilities
                    USING (facid)
                    WHERE facid IN (0,1)
                    ORDER BY surname, firstname
                 ''')
result.toPandas()

#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [ ]:
result = run_sql('''
                   SELECT DISTINCT name, surname, 
                       CASE
                           WHEN surname == 'GUEST' THEN guestcost
                           ELSE membercost
                        END as cost
                   FROM bookings
                   INNER JOIN facilities
                   USING (facid)
                   INNER JOIN members
                   USING (memid)
                   where starttime like "%2012-09-14%"
                   AND (membercost > 30
                   OR guestcost > 30)
                   ORDER BY cost desc
                 ''')
result.toPandas()

#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [ ]:
result = run_sql('''
                    SELECT DISTINCT name, surname, 
                            CASE
                               WHEN surname == 'GUEST' THEN guestcost
                               ELSE membercost
                            END as cost 
                    FROM (SELECT *
                            FROM bookings 
                            INNER JOIN facilities
                            USING (facid)
                            INNER JOIN members
                            USING (memid))
                    WHERE starttime like "%2012-09-14%"
                   AND (membercost > 30
                   OR guestcost > 30)
                   ORDER BY cost desc
                 ''')
result.toPandas()

#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [ ]:
result = run_sql('''
                    SELECT name, memberrevenue + guestrevenue as totalrevenue
                    FROM
                    (SELECT name, SUM(CASE WHEN 'surname' == 'GUEST' THEN 0 ELSE membercost END) as memberrevenue, 
                                SUM(CASE WHEN 'surname' == 'GUEST' THEN guestcost ELSE 0 END) as guestrevenue
                    FROM
                    (SELECT * 
                    FROM bookings
                    LEFT JOIN facilities
                    USING (facid)
                    ORDER BY starttime)
                    GROUP BY name)
                    ORDER BY memberrevenue + guestrevenue
                 ''')
result.toPandas()